# 文字情感分析

在這個範例中，我們使用 Azure 的 Text Analytics API 來完成文字情感分析的功能。  
給定一段文字，我們可以由文字分析的功能中，分析出這段文字中所包含的情感是屬於「正向」(positive)，  
「中立」(neutral)，或是「負向」(negative)。

文字情感分析有許多的用途，例如旅館可以用這個功能分析使用者的回饋，過濾出負向的意見，並針對負向意見做特別的處理。  
又例如基金公司可以分析最近新聞，或是社群網站文章中所透露的情感，藉此預測未來股市的走向。

參考資料:  
- [文字分析API的說明](https://azure.microsoft.com/zh-tw/services/cognitive-services/text-analytics/)
- [文字分析API的簡介](https://docs.microsoft.com/zh-tw/azure/cognitive-services/text-analytics/overview)
- [文字分析API文件 (英文)](https://westus.dev.cognitive.microsoft.com/docs/services/TextAnalytics-v3-0-Preview-1/operations/Sentiment)
- [文字分析API範例](https://github.com/MicrosoftDocs/azure-docs.zh-tw/blob/master/articles/cognitive-services/text-analytics/quickstarts/python.md)
- [文字分析API所支援的語言](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/language-support)




In [6]:
import requests
import json

## 新增服務

在 Azure 的入口網站中，新增 Text Analystics 的服務 (見下圖)。並完成相關設定。

![](textapi1.png)

進入 Text Analystics 服務的設定頁面，點選金鑰與端點 (下圖 (1))，複製金鑰 (下圖(2))，  
以及端點 (下圖(3))，並貼到底下的程式碼中。

![](textapi2.png)

In [7]:
#將上面的金鑰貼入單引號中
subscription_key = ''
#端點後要加上 /text/analytics/v3.0-preview.1/sentiment 的字串，代表我們要使用情感分析的雲端服務
text_sentiment_api_url = 'https://<端點>/text/analytics/v3.0-preview.1/sentiment'

In [16]:
#在檔頭填入金鑰
headers = {'Ocp-Apim-Subscription-Key': subscription_key}



底下我們使用 [hotels.com](https://tw.hotels.com/) 上面，顧客對旅館的評論的文章，來做為分析的對象。  

連上 hotels.com，並隨便選一間旅館。  

![](textapi3.png)

在旅館的首頁的右方，點選「評論」的超連結。  

![](textapi4.png)

隨便複製兩條顧客意見，並貼到下方的程式碼中。  

![](textapi5.png)

In [17]:
body = {"documents": []}
#底下的程式碼中，language要設為 zh-hant，這個設定是告訴微軟，我們的語言是繁體中文。
#text的部分則是使用者的意見。
body['documents'].append(
    {
        'language': 'zh-hant',
        'id':'1',
        'text':'隔音較差，睡眠較淺的人容易被隔壁或走廊的聲音吵醒，也會聽到附近浴室傳出來的聲響。'
    }
)
body['documents'].append(
    {
        'language': 'zh-hant',
        'id':'2',
        'text':'這次入住很特別 是自助辦理入房 在櫃檯停了一下時間有飯店人員協助操作 但訂房時有寫需求希望可以安排高樓層的房間 卻只安排在3樓 不是很好 但房間滿舒適的 熱水很熱 水柱很強 吹風機雖然小但是風很大 而且隔音也還算不錯。'
    }
)

print(body)


{'documents': [{'language': 'zh-hant', 'id': '1', 'text': '隔音較差，睡眠較淺的人容易被隔壁或走廊的聲音吵醒，也會聽到附近浴室傳出來的聲響。'}, {'language': 'zh-hant', 'id': '2', 'text': '這次入住很特別 是自助辦理入房 在櫃檯停了一下時間有飯店人員協助操作 但訂房時有寫需求希望可以安排高樓層的房間 卻只安排在3樓 不是很好 但房間滿舒適的 熱水很熱 水柱很強 吹風機雖然小但是風很大 而且隔音也還算不錯。'}]}


呼叫雲端服務，並取得回傳的結果。

In [18]:
# json 的部分，就是上一個格子中我們填入的意見
response = requests.post(text_sentiment_api_url, headers=headers, json=body)
res = response.json()
print(json.dumps(res))

{"documents": [{"id": "1", "sentiment": "negative", "documentScores": {"positive": 0.37, "neutral": 0.03, "negative": 0.6}, "sentences": [{"sentiment": "negative", "sentenceScores": {"positive": 0.37, "neutral": 0.03, "negative": 0.6}, "offset": 0, "length": 40}]}, {"id": "2", "sentiment": "positive", "documentScores": {"positive": 0.91, "neutral": 0.01, "negative": 0.08}, "sentences": [{"sentiment": "positive", "sentenceScores": {"positive": 0.91, "neutral": 0.01, "negative": 0.08}, "offset": 0, "length": 109}]}], "errors": [], "modelVersion": "2020-04-01"}


In [11]:
# len(res) 可以取得共有幾筆資料回傳。在這個例子中，是三筆
print(len(res))

3


底下的程式碼中，逐一地把每一份回傳的情感分析中所代表的情感都印出來。

In [12]:
docs = res['documents']
print(f'有 {len(docs)} 份回應')
for doc in docs:
    print(f'文章 id: {doc["id"]}')
    print(f'情感 :{doc["sentiment"]}')


有 2 份回應
文章 id: 1
情感 :negative
文章 id: 2
情感 :positive
